In [5]:
from autogen import UserProxyAgent
import os
from dotenv import load_dotenv
from autogen import ConversableAgent,GroupChat,GroupChatManager,AssistantAgent
from autogen.coding import LocalCommandLineCodeExecutor
from google.generativeai.types import HarmCategory, HarmBlockThreshold

load_dotenv()
GOOGLE_API_KEY=os.environ["GOOGLE_API_KEY"]

safety_settings={
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,

    }

In [6]:
llm_config = {"model": "gemini-1.5-flash",
              "api_key": GOOGLE_API_KEY,
              "api_type": "google",
              "safety_settings":safety_settings}

In [ ]:
reply = agent.generate_reply(
    messages=[{"content": "Tell me a joke.", "role": "user"}]
)
print(reply['content'])

In [ ]:
cathy = ConversableAgent(
    name="cathy",
    system_message=
    "Your name is Cathy and you are a stand-up comedian.",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

joe = ConversableAgent(
    name="joe",
    system_message=
    "Your name is Joe and you are a stand-up comedian. "
    "Start the next joke from the punchline of the previous joke.",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

In [ ]:
chat_result = joe.initiate_chat(
    recipient=cathy, 
    message="I'm Joe. Cathy, let's keep the jokes rolling.",
    max_turns=1,
)

In [ ]:
chat_result = joe.initiate_chat(
    cathy, 
    message="I'm Joe. Cathy, let's keep the jokes rolling.", 
    max_turns=2, 
    summary_method="reflection_with_llm",
    summary_prompt="Summarize the conversation",
)

In [ ]:
import pprint
pprint.pprint(chat_result.summary)

In [ ]:
cathy = ConversableAgent(
    name="cathy",
    system_message=
    "Your name is Cathy and you are a stand-up comedian. "
    "When you're ready to end the conversation, say 'I gotta go'.",
    llm_config=llm_config,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "I gotta go" in msg["content"],
)

joe = ConversableAgent(
    name="joe",
    system_message=
    "Your name is Joe and you are a stand-up comedian. "
    "When you're ready to end the conversation, say 'I gotta go'.",
    llm_config=llm_config,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "I gotta go" in msg["content"] or "Goodbye" in msg["content"],
)

In [ ]:
chat_result = joe.initiate_chat(
    recipient=cathy,
    message="I'm Joe. Cathy, let's keep the jokes rolling."
)

In [ ]:
cathy.send(message="What's last joke we talked about?", recipient=joe)

In [ ]:
def get_stock_prices(stock_symbols, start_date, end_date):
    """Get the stock prices for the given stock symbols between
    the start and end dates.

    Args:
        stock_symbols (str or list): The stock symbols to get the
        prices for.
        start_date (str): The start date in the format 
        'YYYY-MM-DD'.
        end_date (str): The end date in the format 'YYYY-MM-DD'.
    
    Returns:
        pandas.DataFrame: The stock prices for the given stock
        symbols indexed by date, with one column per stock 
        symbol.
    """
    import yfinance

    stock_data = yfinance.download(
        stock_symbols, start=start_date, end=end_date
    )
    return stock_data.get("Close")

def plot_stock_prices(stock_prices,filename):
    """Plot the stock prices for the given stock symbols.

    Args:
        stock_prices (pandas.DataFrame): The stock prices for the 
        given stock symbols.
    """
    import matplotlib.pyplot as plt

    plt.figure(figsize=(10, 5))
    for column in stock_prices.columns:
        plt.plot(
            stock_prices.index, stock_prices[column], label=column
                )
    plt.title("Stock Prices")
    plt.xlabel("Date")
    plt.ylabel("Price")
    plt.grid(True)
    plt.savefig(filename)
    plt.close()

In [ ]:
def mongo_to_df():
    import pymongo
    import pandas as pd
    from datetime import datetime
    
    # MongoDB connection details
    client = pymongo.MongoClient("mongodb://localhost:27017/")
    db = client['mydb']  # replace with your database name
    collection = db['sensor']  

    documents = collection.find({}, {
        "deviceId": 1,
        "code": 1,
        "value": 1,
        "creationDate": 1,
        "_id": 0})
    
    df = pd.DataFrame(documents) 

    # Extract the '$date' field from the 'creationDate' column
    df['creationDate'] = df['creationDate'].apply(lambda x: x['$date'] if isinstance(x, dict) else x)

# Convert 'creationDate' to a datetime object
    df['creationDate'] = pd.to_datetime(df['creationDate'])

    return df


def plot_data(device_id, code, plot_type, 
                     start_date=None, end_date=None, 
                     boundary_date=None, before=True, 
                     num_values=None, first=True, 
                     value_threshold=None, min_value=True):
    
    import pandas as pd
    from matplotlib import pyplot as plt
    df = mongo_to_df()
    # Filter DataFrame by device_id and code
    df_filtered = df[(df['deviceId'] == device_id) & (df['code'] == code)]
    
    # Convert 'value' to float
    df_filtered.loc[:, 'value'] = df_filtered['value'].astype(float)
    
    # Convert 'creationDate' to datetime if it's not already
    if df_filtered['creationDate'].dtype != 'datetime64[ns]':
        df_filtered['creationDate'] = pd.to_datetime(df_filtered['creationDate'])

    if plot_type == 'date_range':
        # 1) Plot for a given range of creation dates
        if start_date and end_date:
            df_filtered = df_filtered[(df_filtered['creationDate'] >= pd.to_datetime(start_date)) &
                                      (df_filtered['creationDate'] <= pd.to_datetime(end_date))]

    elif plot_type == 'date_boundary':
        # 2) Plot after/before a given creation date
        if boundary_date:
            if before:
                df_filtered = df_filtered[df_filtered['creationDate'] <= pd.to_datetime(boundary_date)]
            else:
                df_filtered = df_filtered[df_filtered['creationDate'] >= pd.to_datetime(boundary_date)]
    
    elif plot_type == 'recorded_values':
        # 3) Plot for a given range of last/first few recorded values
        if num_values:
            df_filtered = df_filtered.sort_values(by='creationDate', ascending=first).head(num_values)
    
    elif plot_type == 'value_threshold':
        # 4) Plot for a given threshold of value (min/max)
        if value_threshold is not None:
            if min_value:
                df_filtered = df_filtered[df_filtered['value'] >= value_threshold]
            else:
                df_filtered = df_filtered[df_filtered['value'] <= value_threshold]
    
    # Plotting
    plt.scatter(df_filtered['creationDate'], df_filtered['value'])
    plt.xlabel('Creation Date')
    plt.ylabel('Value')
    plt.title(f'Values vs Creation Date for Device ID: {device_id}, Code: {code}')
    plt.grid(True)
    plt.savefig("chart.png")
    plt.close()
    return True

In [ ]:
executor = LocalCommandLineCodeExecutor(
    timeout=60,
    work_dir="coding",
    functions=[plot_data,mongo_to_df],
)

In [ ]:
code_writer_agent_system_message ='''You are a helpful AI assistant.
Solve tasks using your coding and language skills.
In the following cases, suggest python code (in a python coding block) or shell script (in a sh coding block) for the user to execute.
    1. When you need to collect info, use the code to output the info you need, for example, browse or search the web, download/read a file, print the content of a webpage or a file, get the current date/time, check the operating system. After sufficient info is printed and the task is ready to be solved based on your language skill, you can solve the task by yourself.
    2. When you need to perform some task with code, use the code to perform the task and output the result. Finish the task smartly.
Solve the task step by step if you need to. If a plan is not provided, explain your plan first. Be clear which step uses code, and which step uses your language skill.
When using code, you must indicate the script type in the code block. The user cannot provide any other feedback or perform any other action beyond executing the code you suggest. The user can't modify your code. So do not suggest incomplete code which requires users to modify. Don't use a code block if it's not intended to be executed by the user.
If you want the user to save the code in a file before executing it, put # filename: <filename> inside the code block as the first line. Don't include multiple code blocks in one response. Do not ask users to copy and paste the result. Instead, use 'print' function for the output when relevant. Check the execution result returned by the user.
If the result indicates there is an error, fix the error and output the code again. Suggest the full code instead of partial code or code changes. If the error can't be fixed or if the task is not solved even after the code is executed successfully, analyze the problem, revisit your assumption, collect additional info you need, and think of a different approach to try.
When you find an answer, verify the answer carefully. Include verifiable evidence in your response if possible.
You must be able to define the following parameters based on the task.\n 

-device_id (The device id mentioned in the task)\n
-code(the code mentioned in the task)\n
-start_date(start date mentioned in the task)\n
-end_date (end date mentioned in the task)\n
-boundary_date (boundary date mentioned in the task with before or after)\n
-before (whether it is before or after. If it is before, it is assigned True. If it is after, then it is assigned as False)\n 
-num_values (no. of values requested by the task)\n
-first (whether it is the first or last few values requested. If it is first then, it is assigned True)\n
-value_threshold (a threshold value)\n
-min_value (whether the minimum or maximum values for a threshold is requested. If it is a minimum then it is assigned True.)\n
-plot_type (what type of plot should be defined. plot types include date_range(Plot for a given range of creation dates)\n
, date_boundary(Plot after/before a given creation date),\n
recorded_values(Plot for a given range of last/first few recorded values),\n
value_threshold(Plot for a given threshold of value (min/max)))
Reply "TERMINATE" in the end when everything is done.
'''

In [ ]:
code_writer_agent_system_message += executor.format_functions_for_prompt()
print(code_writer_agent_system_message)

In [ ]:
user_proxy = UserProxyAgent(
    name="Admin",
    system_message="Given the task you must provide instructions to code_writer_agent to write the proper code and execute it.",
    code_execution_config={"executor":executor},
    llm_config=llm_config,
    human_input_mode="NEVER",
)

In [ ]:
code_writer_agent = AssistantAgent(
    name="code_writer_agent",
    system_message=code_writer_agent_system_message,
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

In [ ]:
code_writer_agent = ConversableAgent(
    name="code_writer_agent",
    system_message=code_writer_agent_system_message,
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

In [ ]:
code_executor_agent = ConversableAgent(
    name="code_executor_agent",
    llm_config=False,
    code_execution_config={"executor": executor},
    human_input_mode="NEVER",
    default_auto_reply=
    "Please continue. If everything is done, reply 'TERMINATE'.",
)

In [ ]:
groupchat =GroupChat(
    agents=[user_proxy, code_writer_agent],
    messages=[],
    max_round=3,
)

In [ ]:
manager = GroupChatManager(
    groupchat=groupchat, llm_config=llm_config
)

In [ ]:
groupchat_result = user_proxy.initiate_chat(
    manager,
    message="for the deviceId SAG_DPv3_QA3 and code B after the date 2024-08-23, Provide me a plot graph for the values.",
)

In [ ]:
import datetime

today = datetime.datetime.now().date()

In [ ]:
chat_result = code_executor_agent.initiate_chat(
    code_writer_agent,
    message="for the deviceId SAG_DPv3_QA3 and code B between the dates 2024-08-20 and 2024-08-23,Provide me a plot for the values.",
    max_turns=3
)

In [ ]:
import os
from IPython.display import Image
Image(os.path.join("coding", "chart.png"))

In [ ]:
chat_result

In [ ]:
groupchat_result

In [3]:
from custom_agents.viz_agent import agents

In [7]:
user_proxy,manager = agents(llm_config)

In [10]:
groupchat_result = user_proxy.initiate_chat(
    manager,
    message="for the deviceId SAG_DPv3_QA3 and code B between the dates 2024-08-20 and 2024-08-23, Provide me a plot graph for the values.",
)

Admin (to chat_manager):

for the deviceId SAG_DPv3_QA3 and code B between the dates 2024-08-20 and 2024-08-23, Provide me a plot graph for the values.

--------------------------------------------------------------------------------


GroupChat is underpopulated with 2 agents. Consider setting speaker_selection_method to 'round_robin' or allow_repeat_speaker to False, or use direct communication, unless repeated speaker is desired.



Next speaker: code_writer_agent

code_writer_agent (to chat_manager):

```python
# filename: plot_data.py
from functions import plot_data

device_id = "SAG_DPv3_QA3"
code = "B"
start_date = "2024-08-20"
end_date = "2024-08-23"
plot_type = "date_range"

plot_data(device_id, code, plot_type, start_date=start_date, end_date=end_date)
```

I will execute this code to generate the plot for the specified device ID, code, and date range. 

**Please execute the code and provide the plot output.**

TERMINATE 


--------------------------------------------------------------------------------


GroupChat is underpopulated with 2 agents. Consider setting speaker_selection_method to 'round_robin' or allow_repeat_speaker to False, or use direct communication, unless repeated speaker is desired.



Next speaker: Admin


>>>>>>>> EXECUTING CODE BLOCK (inferred language is python)...
Admin (to chat_manager):

exitcode: 0 (execution succeeded)
Code output: 

--------------------------------------------------------------------------------
